In [ ]:
# python -m venv policyengine_venv
# source policyengine_venv/bin/activate
# make sure to select kernel to be in the above environment
# Install ipykernel if not already installed
!pip install ipykernel
!pip install python-dotenv
# Register the environment as a kernel
!python -m ipykernel install --user --name=policyengine_venv --display-name="Python (PolicyEngine)"
# import sys
# print(sys.executable)  # Should show a path inside your policyengine_venv directory
#Install policy engine
!pip install policyengine-uk

In [ ]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the tokens
token = os.environ.get("HUGGING_FACE_TOKEN")

# Now you can use the token
print(f"Token loaded: {token[:4]}...{token[-4:]}")  # Prints first and last 4 chars for verification

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_uk.model_api import *
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
from policyengine_core.parameters import Parameter

import pandas as pd
# LOCAL_enhanced_frs_2022_23="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5"
# sim = Microsimulation(dataset=LOCAL_enhanced_frs_2022_23)
#sim = Microsimulation(dataset="hf://policyengine/policyengine-uk-data/enhanced_frs_2022_23.h5")
 

In [ ]:
# Level 1: 20 GBP pw uplift to child element of UC
start_date= "2029-01-01"
eval_year=2029

def change_tax_parameters(parameters):
    add=20.00*52/12
    parameters.gov.dwp.universal_credit.elements.child.amount.update(
        start=instant(start_date), value=orig_param+add
        )
    parameters.gov.dwp.universal_credit.elements.child.first.higher_amount.update(
        start=instant(start_date), value=orig_param_higher+add
    )
    return parameters


class reform(Reform):
    def apply(self):
        #self.modify_parameters(change_tax_parameters)
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
Microsimulation()
orig_param = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.elements.child.amount(start_date)
orig_param_higher = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.elements.child.first.higher_amount(start_date)
print(orig_param,orig_param_higher )
reformed = Microsimulation(reform=reform)

revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in {eval_year}")
povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)


In [ ]:
#Level1b:  Work allowance increased to 1000 GBP  pa
start_date= "2029-01-01"
eval_year=2029

def change_tax_parameters(parameters):
    add=1000.00/12  # Monthly increase

    parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing.update(
        start=instant(start_date), value=orig_without_housing+add
    )
    parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing.update(
        start=instant(start_date), value=orig_with_housing+add
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
orig_without_housing = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.means_test.work_allowance.without_housing(start_date)
orig_with_housing = baseline.tax_benefit_system.parameters.gov.dwp.universal_credit.means_test.work_allowance.with_housing(start_date)
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calc("gov_spending", eval_year).sum()
print(f"This reform would cost £{revenue / 1e9:.2f}bn in {eval_year}")

povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)



In [ ]:
#Level 1c: Change child limit
start_date= "2026-01-01"
eval_year=2026
def change_tax_parameters(parameters):
    parameters.gov.dwp.universal_credit.elements.child.limit.child_count.update(
        start=instant(start_date), value=inf
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(change_tax_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)
avg_gain_gainer(baseline, reformed, eval_year)


In [ ]:
#Level 2b UNDER 5s premium
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_bonus_for_under5 import reform
eval_year=2029
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
print(reformed.calculate("num_households_with_under5", eval_year).sum())
print(reformed.calculate("universal_credit", eval_year).mean())
print(baseline.calculate("universal_credit", eval_year).mean())
print(f"Number of households with at least one child under 5: {under5_households}")
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)

In [ ]:
#Level 3 ADD 30 HOUR WORK BONUS
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_30hour_bonus_v2 import reform
from eval_functions import povertycalc, avg_gain_gainer
eval_year=2026
baseline = Microsimulation()
NMW = baseline.tax_benefit_system.parameters.gov.hmrc.minimum_wage.OVER_24(eval_year)
print(NMW)
reformed = Microsimulation(reform=reform)
# Debug: Check if UC is actually changing
baseline_uc = baseline.calculate("universal_credit", eval_year)
reformed_uc = reformed.calculate("universal_credit", eval_year)
print("baseline uc:",baseline.calculate("universal_credit", eval_year).mean())
print("reformed uc:",reformed.calculate("universal_credit", eval_year).mean())
revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)

In [ ]:
# Add EITC phase in -- EVALUATION
from policyengine_uk import Microsimulation
from policyengine_uk.reforms.uc_phase_in import reform
from eval_functions import povertycalc, avg_gain_gainer
eval_year=2026
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
# Debug: Check if UC is actually changing
baseline_uc = baseline.calculate("universal_credit", eval_year)
reformed_uc = reformed.calculate("universal_credit", eval_year)
print("baseline uc:",baseline.calculate("universal_credit", eval_year).mean())
print("reformed uc:",reformed.calculate("universal_credit", eval_year).mean())

revenue = reformed.calculate("gov_spending", eval_year).sum() - baseline.calculate("gov_spending", eval_year).sum()
print(f"Cost: £{round(revenue / 1e+9, 1)}bn")
povertycalc(baseline, eval_year, reformed)  
avg_gain_gainer(baseline, reformed, eval_year)



In [ ]:
# Add EITC phase in -- GRAPH
from policyengine_uk import Simulation, Microsimulation
from policyengine_uk.reforms.uc_phase_in import reform
import pandas as pd
import plotly.express as px
situation = {
    "people": {
        "person": {
            "age": 26,
        },
        "child":{
            "age": 5,
        },
    },
    "households": {
        "household": {
            "members": ["person","child"],
            "rent": 1_000,
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 10_000,
            "count": 301,
        }
    ]]
}

baseline = Simulation(
    situation=situation,
)

reformed = Simulation(
    situation=situation,
    reform=reform,
)
df = pd.DataFrame()

year = 2025
df["Employment income"] = baseline.calculate("employment_income", year, map_to="household")
df["UC (baseline)"] = baseline.calculate("universal_credit", year, map_to="household")
df["UC (reformed)"] = reformed.calculate("universal_credit", year, map_to="household")
# df["Net income (baseline)"] = baseline.calculate("household_net_income", year, map_to="household")
# df["Net income (reformed)"] = reformed.calculate("household_net_income", year, map_to="household")
# df["Marginal tax rate (baseline)"] = baseline.calculate("marginal_tax_rate", year, map_to="household")
# df["Marginal tax rate (reformed)"] = reformed.calculate("marginal_tax_rate", year, map_to="household")

fig = px.line(
    df,
    x="Employment income",
    y=df.columns,
    labels={
        "value": "Universal Credit",
        "Employment income": "Employment income",
    },
    title="Universal Credit taper reform",
)
fig.update_layout(xaxis_range=[0, 10_000])
fig.show()


/Users/helenauta.bolt/Documents/GitHub/policyengine-uk/policyengine_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from policyengine_uk.system import system
import plotly.express as px
from policyengine_core.charts import format_fig
from policyengine_uk import Simulation

simulation = Simulation(
    situation=dict(
        people=dict(
            person=dict(age=30),
            child1=dict(age=8),
        ),
        households=dict(
            household=dict(
                members=["person", "child1"],
            )
        ),
        #benunit=dict(benunit=dict(benunit_count_children=2)),
        axes=[
            [
                dict(
                    name="employment_income",
                    min=0,
                    max=20000,
                    count=3_00,
                )
            ]
        ],
    ),
)

import plotly.express as px
import pandas as pd
from policyengine_core.charts import format_fig

df = pd.DataFrame(
    {
        "Employment income": simulation.calculate("employment_income", map_to="household"),
        "Universal credit": simulation.calculate("universal_credit", map_to="household"),
        "household_net_income": simulation.calculate("household_net_income", map_to="household"),
        "household_benefits": simulation.calculate("household_benefits", map_to="household"),

    }
)
 
fig = px.line(
    df,
    x="Employment income",
    y=["Universal credit","household_net_income","household_benefits"],
    title="UC vs employment income",
    labels={
        "value": "Income  (£)",
        "variable": "Income type",
    },
)
fig.update_layout(xaxis_range=[0, 20_000])

format_fig(fig)

In [ ]:
from policyengine_uk.system import system
import plotly.express as px
from policyengine_core.charts import format_fig
from policyengine_uk import Simulation, Microsimulation

import pandas as pd


situation = {
    "people": {
        "person": {
            "age": 26,
        },
    },
    "households": {
        "household": {
            "members": ["person"],
            "rent": 6_000,
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 5_000,
            "count": 301,
        }
    ]]
}

baseline = Simulation(
    situation=situation,
)

# Calculate the values
df = pd.DataFrame(
    {
        #"Hours":simulation.calculate("hours_worked"),
        "Employment income": simulation.calculate("household_gross_income")  ,
        "Universal Credit": simulation.calculate("universal_credit"),
        "Household net income": simulation.calculate("household_net_income"),
        "Household benefits": simulation.calculate("household_benefits"),
    }
)

# Create the plot
fig = px.line(
    df,
    x="Employment income",
    y=["Universal Credit", "Household net income", "Household benefits"],
    title="Universal Credit vs Employment Income (Single Parent, 2 Children)",
    labels={
        "value": "Income (£/year)",
        "variable": "Income type",
        "Employment income": "Employment Income (£/year)"
    },
)

format_fig(fig)

# # Optional: Print some key values to debug
# print("First few rows of data:")
# print(df.head(10))
# print(f"\nMax UC amount: £{df['Universal Credit'].max():.2f}")
# print(f"UC becomes zero at employment income: £{df[df['Universal Credit'] <= 0]['Employment income'].min():.2f}")

fig.show()

In [ ]:
from policyengine_uk import Simulation, Microsimulation
import plotly.express as px
import pandas as pd

situation = {
    "people": {
        "person": {
            "age": 26,
        },
    },
    "households": {
        "household": {
            "members": ["person"],
            "rent": 6_000,
        }
    },
    "axes": [[
        {
            "name": "employment_income",
            "min": 0,
            "max": 30_000,
            "count": 301,
        }
    ]]
}

baseline = Simulation(
    situation=situation,
)

reformed = Simulation(
    situation=situation,
)


df = pd.DataFrame()

year = 2025
df["Employment income"] = baseline.calculate("employment_income", year, map_to="household")
df["UC (baseline)"] = baseline.calculate("universal_credit", year, map_to="household")
df["Net income (baseline)"] = baseline.calculate("household_net_income", year, map_to="household")
df["Marginal tax rate (baseline)"] = baseline.calculate("marginal_tax_rate", year, map_to="household")


fig = px.line(
    df,
    x="Employment income",
    y=df.columns,
    labels={
        "value": "Universal Credit",
        "Employment income": "Employment income",
    },
    title="Universal Credit taper reform",
)

format_fig(fig)

In [ ]:
from policyengine_uk.model_api import *


# I'm including the old version of this variable just so we can compare here.

class uc_income_reduction_old(Variable):
    value_type = float
    entity = BenUnit
    label = "reduction from income for Universal Credit"
    definition_period = YEAR
    unit = GBP
    defined_for = "is_uc_eligible"

    def formula(benunit, period, parameters):
        p = parameters(period).gov.dwp.universal_credit.means_test
        earned_income = benunit("uc_earned_income", period)
        earned_income_reduction = p.reduction_rate * earned_income
        unearned_income_reduction = benunit("uc_unearned_income", period)
        maximum_credit = benunit("uc_maximum_amount", period)
        total_reduction = earned_income_reduction + unearned_income_reduction
        return min_(
            maximum_credit,
            total_reduction,
        )
    

young_reduction_rate = Parameter(
    "young_reduction_rate",
    data={
        "2025-01-01": 0.4,
        "2026-01-01": 0.35,
    }
)

class uc_income_reduction(Variable):
    value_type = float
    entity = BenUnit
    label = "reduction from income for Universal Credit"
    definition_period = YEAR
    unit = GBP
    defined_for = "is_uc_eligible"

    def formula(benunit, period, parameters):
        p = parameters(period).gov.dwp.universal_credit.means_test
        earned_income = benunit("uc_earned_income", period)

        # NEW
        is_young = benunit("eldest_adult_age", period) < 30
        reduction_rate = where(
            is_young,
            p.young_reduction_rate,
            p.reduction_rate,
        )
        # END NEW

        earned_income_reduction = reduction_rate * earned_income
        unearned_income_reduction = benunit("uc_unearned_income", period)
        maximum_credit = benunit("uc_maximum_amount", period)
        total_reduction = earned_income_reduction + unearned_income_reduction
        return min_(
            maximum_credit,
            total_reduction,
        )

def modify_parameters(parameters: ParameterNode):
    if "young_reduction_rate" not in parameters.gov.dwp.universal_credit.means_test.children:
        parameters.gov.dwp.universal_credit.means_test.add_child(
            "young_reduction_rate",
            young_reduction_rate
        )
    return parameters

class uc_taper_reform(Reform):
    def apply(self):
        self.update_variable(uc_income_reduction)
        self.modify_parameters(modify_parameters)
